In [2]:
import pandas as pd
from recordlinkage.datasets import load_febrl3, load_febrl4

Load FEBRL

In [13]:
PATH_DATA = "../data/"

# FEBRL 3 loading
df_febrl3, links_febrl3 = load_febrl3(return_links=True)  # use this for load_febrl4
FILE_NAME_FEBRL3 = "febrl3_reproduced.csv"

# FEBRL 4 loading
df_febrl4_a, df_febrl4_b, links_febrl4 = load_febrl4(return_links=True)
df_febrl4 = df_febrl4_a.append(df_febrl4_b)  # use for load_febrl4
FILE_NAME_FEBRL4 = "febrl4_reproduced.csv"

PATH_FEBRL3 = PATH_DATA + FILE_NAME_FEBRL3
PATH_FEBRL4 = PATH_DATA + FILE_NAME_FEBRL4

/tmp/ipykernel_2007/1896272727.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_febrl4 = df_febrl4_a.append(df_febrl4_b)  # use for load_febrl4


Preprocess FEBRL

In [14]:
def febrl_preprocess(df_febrl, links_febrl):
    df_febrl["date_of_birth"] = pd.to_datetime(df_febrl["date_of_birth"], errors="coerce")
    df_febrl["day"] = df_febrl["date_of_birth"].dt.strftime("%d")
    df_febrl["month"] = df_febrl["date_of_birth"].dt.strftime("%m")
    df_febrl["year"] = df_febrl["date_of_birth"].dt.strftime("%Y")

    df_febrl["postcode"] = df_febrl["postcode"].fillna("0000")
    df_febrl["postcode"] = df_febrl["postcode"].astype(int)

    df_febrl["street_number"] = df_febrl["street_number"].fillna("0")
    df_febrl["street_number"] = df_febrl["street_number"].astype(int)

    df_febrl = df_febrl.drop(["soc_sec_id", "date_of_birth"], axis=1)

    for col in ["surname", "given_name", "address_1", "address_2", "day", "month"]:
        df_febrl[col] = df_febrl[col].fillna("")
        df_febrl[col] = df_febrl[col].astype(str)

    df_febrl["match_id"] = [-1] * len(df_febrl)

    for i in range(len(links_febrl)):
        k0 = links_febrl[i][0]
        k1 = links_febrl[i][1]
        df_febrl.loc[k0, "match_id"] = i
        df_febrl.loc[k1, "match_id"] = i

    return df_febrl

Save FEBRL to csv

In [15]:
df_febrl3_preprocessed = febrl_preprocess(df_febrl3, links_febrl3)
df_febrl3_preprocessed.to_csv(PATH_FEBRL3, index=True)

df_febrl4_preprocessed = febrl_preprocess(df_febrl4, links_febrl4)
df_febrl4_preprocessed.to_csv(PATH_FEBRL4, index=True)